In [1]:
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA
import pymongo
from pymongo import MongoClient

In [2]:
# Connection to mongo
client = MongoClient('mongodb+srv://<username>:<password>@cluster0.l3pqt.mongodb.net/MSA?retryWrites=true&w=majority')
# Select database
db = client['MSA']
# see list of collections
client.MSA.list_collection_names()

['arima_LaborParticipation_pred_score',
 'CPI_RegionUrban_raw',
 'BusinessApplications_clean',
 'arima_GDP_2019_2024',
 'employment_prediction',
 'Employment_clean',
 'CPI_RegionUrban_clean',
 'population_predicted_2024',
 'employment_clean',
 'LaborForce_clean',
 'GDP_raw',
 'unemployment_ROC_2024',
 'employment_raw',
 'OES_raw',
 'Population_clean',
 'unemployment_predicted_2024',
 'arima_pop_pred_score',
 'msa_codes',
 'arima_unem_pred_score',
 'population_raw',
 'arima_emp_pred_score',
 'LaborForce_raw',
 'arima_cpi_pred_score',
 'UnemploymentRate_clean',
 'Employment_raw',
 'arima_2024_ROC_score_total',
 'regions_divisions',
 'LaborParticipation_clean',
 'unemployment_raw',
 'AnnualMeanWage_clean',
 'GDP_clean',
 'UnemploymentRate_raw',
 'Predicted_2024_ROC_rank_total',
 'arima_GDP_pred_score',
 'Predicted_2024_ROC_rank_total2',
 'GDP_predicted_2024']

In [3]:
df = pd.read_csv('employment_MSA.csv')
print(df.dtypes)
df

GeoFips     int64
GeoName    object
2001        int64
2002        int64
2003        int64
2004        int64
2005        int64
2006        int64
2007        int64
2008        int64
2009        int64
2010        int64
2011        int64
2012        int64
2013        int64
2014        int64
2015        int64
2016        int64
2017        int64
2018        int64
2019        int64
dtype: object


,GeoFips,GeoName,2001,2002,2003,2004,2005,2006,2007,2008,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,10180,"Abilene, TX (Metropolitan Statistical Area)",89511,90338,91132,91998,92937,94097,96484,99264,...,96809,98193,99843,101199,102316,102611,102601,102786,104536,107405
1,10420,"Akron, OH (Metropolitan Statistical Area)",396954,396663,396391,404591,412097,417595,424012,424079,...,402554,407811,412049,414502,421225,426047,427394,428034,430261,433800
2,10500,"Albany, GA (Metropolitan Statistical Area)",78011,77100,78282,78126,80177,80621,81153,80663,...,77793,79189,79388,79694,80577,81418,81714,81872,82431,83286
3,10540,"Albany-Lebanon, OR (Metropolitan Statistical A...",54749,54136,53710,54697,57088,58162,59546,59473,...,55562,55341,55676,56161,57670,58997,59661,61777,63422,64277
4,10580,"Albany-Schenectady-Troy, NY (Metropolitan Stat...",516432,514437,518106,524002,530740,534257,540904,543929,...,528095,531541,537634,543162,548362,556276,563056,569909,575772,579135
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
379,49420,"Yakima, WA (Metropolitan Statistical Area)",112685,112734,114189,113223,114927,117779,118536,121722,...,119123,120041,124847,125387,127305,128600,130601,132743,134982,136556
380,49620,"York-Hanover, PA (Metropolitan Statistical Area)",211514,211138,211658,217006,223488,226836,231790,232142,...,221987,224717,225936,227264,229563,231735,235643,237721,240494,243152
381,49660,"Youngstown-Warren-Boardman, OH-PA (Metropolita...",311150,305452,302591,301803,303520,302814,301433,295953,...,280982,285225,287302,287503,287915,287937,288202,284315,284853,283700
382,49700,"Yuba City, CA (Metropolitan Statistical Area)",61499,62404,62573,63898,64648,66167,67736,65442,...,62526,62563,63966,65465,66816,68645,69903,71388,73096,75581


In [4]:
df.drop(columns='GeoName', inplace=True)
df.head()

,GeoFips,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,10180,89511,90338,91132,91998,92937,94097,96484,99264,97269,96809,98193,99843,101199,102316,102611,102601,102786,104536,107405
1,10420,396954,396663,396391,404591,412097,417595,424012,424079,404521,402554,407811,412049,414502,421225,426047,427394,428034,430261,433800
2,10500,78011,77100,78282,78126,80177,80621,81153,80663,78714,77793,79189,79388,79694,80577,81418,81714,81872,82431,83286
3,10540,54749,54136,53710,54697,57088,58162,59546,59473,56122,55562,55341,55676,56161,57670,58997,59661,61777,63422,64277
4,10580,516432,514437,518106,524002,530740,534257,540904,543929,535314,528095,531541,537634,543162,548362,556276,563056,569909,575772,579135


In [5]:
df.rename(columns={'GeoFips': 'CBSA'}, inplace=True)

In [6]:
# for loop to predict 2019 so percent error score can be calculated
predictions = []
for i in range(0,384):
    y = df.iloc[i, 1:-1].values
    series = pd.Series(y, dtype='int')
    model = ARIMA(series, order=(2, 1, 1))
    model_fit = model.fit()
    pred = model_fit.forecast(1)
    predictions.append(pred.values.tolist()[0])
df['2019_pred'] = predictions

C:\Users\Joshua\anaconda3\envs\mlenv\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
C:\Users\Joshua\anaconda3\envs\mlenv\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
C:\Users\Joshua\anaconda3\envs\mlenv\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
C:\Users\Joshua\anaconda3\envs\mlenv\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
C:\Users\Joshua\anaconda3\envs\mlenv\lib\site-packages\statsmodels\base\model.py:568: Convergen

C:\Users\Joshua\anaconda3\envs\mlenv\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
C:\Users\Joshua\anaconda3\envs\mlenv\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
C:\Users\Joshua\anaconda3\envs\mlenv\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
C:\Users\Joshua\anaconda3\envs\mlenv\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
C:\Users\Joshua\anaconda3\envs\mlenv\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
C:\Users\J

C:\Users\Joshua\anaconda3\envs\mlenv\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
C:\Users\Joshua\anaconda3\envs\mlenv\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
C:\Users\Joshua\anaconda3\envs\mlenv\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
C:\Users\Joshua\anaconda3\envs\mlenv\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
C:\Users\Joshua\anaconda3\envs\mlenv\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
C:\Users\J

C:\Users\Joshua\anaconda3\envs\mlenv\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
C:\Users\Joshua\anaconda3\envs\mlenv\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
C:\Users\Joshua\anaconda3\envs\mlenv\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
C:\Users\Joshua\anaconda3\envs\mlenv\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
C:\Users\Joshua\anaconda3\envs\mlenv\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
C:\Users\J

In [7]:
df.head()

,CBSA,2001,2002,2003,2004,2005,2006,2007,2008,2009,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2019_pred
0,10180,89511,90338,91132,91998,92937,94097,96484,99264,97269,...,98193,99843,101199,102316,102611,102601,102786,104536,107405,105398.166432
1,10420,396954,396663,396391,404591,412097,417595,424012,424079,404521,...,407811,412049,414502,421225,426047,427394,428034,430261,433800,430263.072932
2,10500,78011,77100,78282,78126,80177,80621,81153,80663,78714,...,79189,79388,79694,80577,81418,81714,81872,82431,83286,82429.821011
3,10540,54749,54136,53710,54697,57088,58162,59546,59473,56122,...,55341,55676,56161,57670,58997,59661,61777,63422,64277,63930.339815
4,10580,516432,514437,518106,524002,530740,534257,540904,543929,535314,...,531541,537634,543162,548362,556276,563056,569909,575772,579135,575901.992367


In [8]:
# calculate percent error for each linear regression
df['Percent_Error'] = (df['2019'] - df['2019_pred'])/df['2019']*100

# set values to absolute in percent_error column
df['Percent_Error'] = df['Percent_Error'].abs()

# print mean of a percent_error
mean_percent_error = df['Percent_Error'].mean()
print(f'Average percent error for ARIMA predicting 2019 employment is: {round(mean_percent_error,3)}%' )
df.head()

Average percent error for ARIMA predicting 2019 empoloyment is: 1.018%


,CBSA,2001,2002,2003,2004,2005,2006,2007,2008,2009,...,2012,2013,2014,2015,2016,2017,2018,2019,2019_pred,Percent_Error
0,10180,89511,90338,91132,91998,92937,94097,96484,99264,97269,...,99843,101199,102316,102611,102601,102786,104536,107405,105398.166432,1.868473
1,10420,396954,396663,396391,404591,412097,417595,424012,424079,404521,...,412049,414502,421225,426047,427394,428034,430261,433800,430263.072932,0.815336
2,10500,78011,77100,78282,78126,80177,80621,81153,80663,78714,...,79388,79694,80577,81418,81714,81872,82431,83286,82429.821011,1.027999
3,10540,54749,54136,53710,54697,57088,58162,59546,59473,56122,...,55676,56161,57670,58997,59661,61777,63422,64277,63930.339815,0.539322
4,10580,516432,514437,518106,524002,530740,534257,540904,543929,535314,...,537634,543162,548362,556276,563056,569909,575772,579135,575901.992367,0.558248
